<a href="https://colab.research.google.com/github/lucarenz1997/machine-learning-1/blob/main/NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NCF (Neural Collaborative Filtering) Train model
** Authors **: Rafaella and Luca
## Provisorischer Data Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Loading the dataset
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Recommender/train.csv")


import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'google'

## NCF (Neural Collaborative Filtering) Train model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# Mapping von user_id und media_id zu Indizes für das Modell
user_map = {u: i for i, u in enumerate(df_encoded['user_id'].unique())}
media_map = {m: i for i, m in enumerate(df_encoded['media_id'].unique())}

df_encoded['user_idx'] = df_encoded['user_id'].map(user_map)
df_encoded['media_idx'] = df_encoded['media_id'].map(media_map)

# Trainings- und Testdaten aufteilen
train, test = train_test_split(df_encoded, test_size=0.2, random_state=42)

# Embedding-Dimensionen
embedding_dim = 32

# NCF-Modell definieren
user_input = Input(shape=(1,))
media_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=len(user_map), output_dim=embedding_dim)(user_input)
media_embedding = Embedding(input_dim=len(media_map), output_dim=embedding_dim)(media_input)

user_flatten = Flatten()(user_embedding)
media_flatten = Flatten()(media_embedding)

concat = Concatenate()([user_flatten, media_flatten])
dense = Dense(64, activation='relu')(concat)
dense = Dense(32, activation='relu')(dense)
output = Dense(1, activation='sigmoid')(dense)  # Vorhersage zwischen 0 und 1

# Modell kompilieren
model = Model(inputs=[user_input, media_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Trainingsdaten
X_train = [train['user_idx'].values, train['media_idx'].values]
y_train = train['is_listened'].values

# Testdaten
X_test = [test['user_idx'].values, test['media_idx'].values]
y_test = test['is_listened'].values

# Modell trainieren
model.fit(X_train, y_train, epochs=5, batch_size=512, validation_data=(X_test, y_test))


genre_id                 category
ts_listen          datetime64[ns]
media_id                 category
album_id                 category
context_type             category
release_date       datetime64[ns]
platform_name            category
platform_family          category
media_duration              int64
listen_type              category
user_gender                  bool
user_id                  category
artist_id                category
user_age                    int64
is_listened                  bool
dtype: object
